# 교차 검증과 하이퍼파라미터 튜닝

**데이터 준비 후 타겟 열과 특성 열을 분리하여 Numpy 배열로 변환하기**

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

**훈련 세트와 테스트 세트 나누기**

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

**훈련 세트를 다시 한 번 훈련 세트(=sub)와 검증 세트(=val)로 나눈 뒤 크기 확인하기**

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


**훈련 세트(=sub)와 검증 세트(=val)를 결정 트리 모델에 학습시킨 뒤 정확도 확인하기**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


**결정 트리 모델 교차 검증 후 평균 점수 확인하기**

In [ ]:
from sklearn.model_selection import cross_validate
import numpy as np

scores = cross_validate(dt, train_input, train_target)
print(np.mean(scores['test_score']))

0.855300214703487


**훈련 세트를 섞어 10-폴드 교차 검증한 뒤 평균 점수 확인하기**

In [ ]:
from sklearn.model_selection import StratifiedKFold

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


**그리드 서치를 위해 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리 형태로 만들기**

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

**그리드 서치 객체 생성 후 5-폴드 교차 검증 수행하기**

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})